# Import the libraries that are needed

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Initialize train and test dataset

In [ ]:
targetdata =  #data that we want to predict
dim_targetdata =  #number of values that need to be predicted per input point
modinfo =  #input data for the model
dim_modinfo =#number of input parameters per input point
train_size =  #number of datapoints in the train dataset
test_size = #number of datapoints in the test dataset
dim_latent_space = #dimension of the latent space representation

In [ ]:
indexen = range(len(targetdata))

#X_trainges, X_testges, training_yges, y_testges = train_test_split(modinfo, verschilgeschaald, train_size=2500, test_size=2400, random_state=333)
verschilgeschaald_train, verschilgeschaald_test, indexen_trainges, indexen_testges = train_test_split(targetdata, indexen, train_size=train_size, test_size=test_size, random_state=333)


#Creating the encoder-decoder

In [ ]:
X = np.array(verschilgeschaald_train).reshape(len(verschilgeschaald_train), dim_targetdata)

params_input = keras.Input(shape=(dim_targetdata)) #creating the input layer
encoder = tf.keras.layers.Dense(dim_latent_space)(params_input) 
decoder = tf.keras.layers.Dense(dim_targetdata)(encoder)

model = keras.Model(inputs=params_input,outputs=decoder)
model.compile(optimizer='adam', loss='mse')
model.summary()
hist = model.fit(X,X,epochs=1000,validation_split=0.2,batch_size=32)

# Extract the hidden states

In [ ]:
Xen = np.array(targetdata).reshape(len(targetdata), 99)
layer_output=model.get_layer('dense').output #'dense' is the name of the encoder layer, if the algorithm is performed multiple times, the name changes to dense_number
intermediate_model=tf.keras.models.Model(inputs=params_input,outputs=layer_output)
tussen=intermediate_model.predict(Xen) # = hidden vectors
np.savetxt("hidden.csv", tussen, delimiter=",")

# Make decoder model

In [ ]:
layer_hidden = model.get_layer('dense').output #'dense' is the name of the encoder layer, if the algorithm is performed multiple times, the name changes to dense_number
decoder_model = tf.keras.models.Model(inputs=layer_hidden,outputs=decoder)

# Train the neural network to predict the hidden states

In [ ]:
#3 layers with 100, 20 and dim_latent_space neurons, as dim_latent_space changes the number of neurons in the layers might need to be changed to obtain an optimal model
modinfo_train = modinfo[indexen_trainges]
tussen_train = tussen[indexen_trainges]
X= np.array(modinfo_train).reshape(len(modinfo_train), dim_modinfo)
Y = np.array(tussen_train).reshape(len(tussen_train), dim_latent_space)

params_input = keras.Input(shape=(dim_modinfo))
lay1 = tf.keras.layers.Dense(100)(params_input)
lay2 = tf.keras.layers.Dense(20)(lay1)
lay3 = tf.keras.layers.Dense(dim_latent_space)(lay2)

modelnn = keras.Model(inputs=params_input,outputs=lay3)
modelnn.compile(optimizer='adam', loss='mse')
modelnn.summary()
hist = modelnn.fit(X,Y,epochs=1000,validation_split=0.2,batch_size=32)

# Use the decoder to go back to the full representation 

In [ ]:
Xtot = np.array(modinfo).reshape(len(modinfo), dim_modinfo)
ouputnn = np.array(modelnn.predict(Xtot))
ouputnnfr = np.array(ouputnn).reshape(len(ouputnn),dim_latent_space)
predictie = decoder_model.predict(ouputnnges)